In [14]:
####################################################################################################
#  This will read in a model number and output the stats for that specific model.
# It will calculate the KGE (Kling-Gupta Efficiency), RMSE, and Residuals for the model.
# However, given that some of these models have different regional models that make up the main model,
# it will need to be separated into different regions. But only for the models that do have regional models.
# The stats will be outputted to a csv file. To be used in the future for analysis.
####################################################################################################

import sys
import numpy as np
import pandas as pd
import geopandas as gpd

In [15]:
test = pd.DataFrame(data=
                    {'obs': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                     'sim': [1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9, 10.1]})
test

,obs,sim
0,1,1.1
1,2,2.2
2,3,3.3
3,4,4.4
4,5,5.5
5,6,6.6
6,7,7.7
7,8,8.8
8,9,9.9
9,10,10.1


In [16]:
# Caclulate the RMSE for a predicted and observed dataset
# Assume that the observed and predicted datasets are pandas dataframes with the same length
def rmse(observed, predicted):
    return np.sqrt(np.mean((observed - predicted) ** 2))

In [17]:
# Calculate the KGE for a predicted and observed dataset
# Assume that the observed and predicted datasets are pandas dataframes with the same length
def kge(observed, predicted):
    # Calculate mean and standard deviation of observed and predicted datasets
    obs_mean = observed.mean()
    sim_mean = predicted.mean()
    obs_std = observed.std()
    sim_std = predicted.std()

    # Calculate the correlation coefficient between the observed and predicted datasets
    r = np.corrcoef(observed, predicted)[0, 1] # [0, 1] is the correlation between the two datasets in the matrix

    # Calculate the bias and variability between the observed and predicted datasets
    beta = sim_mean / obs_mean
    alpha = sim_std / obs_std

    # Calculate the Kling-Gupta Efficiency
    return 1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)